In [ ]:
import numpy as np
import pandas as pd

In [ ]:
shark_df = pd.read_csv('./shark_final_df.csv')
shark_df.head(5)

In [ ]:
shark_df.activity.value_counts()

# Function Block (Risk Calculation)

In [ ]:
# must be updated relative to the cleaned dataframe
coastline_lengths = {
    'USA': 19924,
    'Australia': 25760,
    'South Africa': 2798,
    'Bahamas': 3542,
    'New Zealand': 15134,
    'Mexico': 9330,
    'Brazil': 7491,
    'New Caledonia': 2254,
    'French Polynesia': 2525,
    'Egypt': 2450}

def risk_country(client_country):
    
    top_10_countries = shark_df.country.value_counts().head(10)
    
    # attack risk for the client country = number of attacks in the client country divided by the coastline length 
    attack_risk = round(top_10_countries[client_country] / coastline_lengths[client_country], 3)     
    
    # number of attacks in the client country that were fatal
    number_of_fatal_attacks = len(shark_df[(shark_df.country == client_country) & (shark_df.fatal == 'yes')])
    
    # number of fatal attacks divided by the total number of shark attacks in that country
    fatality_risk = number_of_fatal_attacks / top_10_countries[client_country]
    
    return fatality_risk * attack_risk


def risk_season(client_season):
    
    seasons = shark_df.season.value_counts()
    
    number_of_fatal_attacks = len(shark_df[(shark_df.season == client_season) & (shark_df.fatal == 'yes')])
    fatality_risk = number_of_fatal_attacks / seasons[client_season]
    
    return fatality_risk


def risk_activity(client_activity):
    
    top_10_risks = shark_df.activity.value_counts().head(10)
    
    number_of_fatal_attacks = len(shark_df[(shark_df.activity == client_activity) & (shark_df.fatal == 'yes')])
    fatality_risk = number_of_fatal_attacks / top_10_risks[client_activity]
    
    return fatality_risk


#def risk_age(client_age):
    
    #number_of_fatal_attacks = len(shark_df[(shark_df.age == client_age) & (shark_df.fatal == 'yes')])
    #fatality_risk = number_of_fatal_attacks / shark_df.age.value_counts()[client_age]
    
    #return fatality_risk


def risk_sex(client_sex):
    
    sexs = shark_df.sex.value_counts()
    
    number_of_fatal_attacks = len(shark_df[(shark_df.sex == client_sex) & (shark_df.fatal == 'yes')])
    fatality_risk = number_of_fatal_attacks / sexs[client_sex]
    
    return fatality_risk


def assign_insurance(client_country, client_season, client_activity, client_sex): #client_age
    
    risk = risk_country(client_country) * risk_season(client_season) * risk_activity(client_activity) * risk_sex(client_sex) #* risk_age(client_age)
    
    print(risk)

    if risk >= float(percentiles.loc[0.25]):
        return 'high'
    
    elif risk > float(percentiles.loc[0.15]):
        return 'medium'
    
    else:
        return 'low'

In [ ]:
# assign a probability to everyone in the dataset
# take the 25% percentile of that as low, 50% percentile as medium and 75% percentile as high

shark_df['country_risk'] = shark_df['country'].apply(lambda country: risk_country(country))
shark_df['season_risk'] = shark_df['season'].apply(lambda season: risk_season(season))
shark_df['activity_risk'] = shark_df['activity'].apply(lambda activity: risk_activity(activity))
#shark_df['age_risk'] = shark_df['age'].apply(lambda age: risk_age(age))
shark_df['sex_risk'] = shark_df['sex'].apply(lambda sex: risk_sex(sex))

shark_df['total_risk'] = shark_df['country_risk'] * shark_df['season_risk'] * shark_df['activity_risk'] * shark_df['sex_risk'] #* shark_df['age_risk']
percentiles = shark_df['total_risk'].quantile([0.15, 0.25]).apply(lambda x: f"{x:.4e}")
print(percentiles.loc[0.15], percentiles.loc[0.25])

In [ ]:
# User input function
def client_entries():
    client_country = input('Please enter the country you travel to: ').strip()
    client_activity = input('Please enter your activity: ').strip()
    #client_age = input('Please enter your age: ').strip()
    client_sex = input('Please enter your sex: ').strip()
    client_season = input('Please enter the season of your trip: ').strip()
    return client_country, client_season, client_activity, client_sex #client_age

In [ ]:
assign_insurance(*client_entries())